# CSIKit Visualization Module Demo
## Introduction

This Jupyter notebook outlines and demonstrates the functionality offered with the `visualization` module, implemented
by [twiegel-dev](https://github.com/tweigel-dev).

Currently, this module is only compatible with Intel 5300 .dat-style CSI,
however other formats will be supported in time. If you've got any questions, amendments, feature suggestions, or any
other comments, please reach out in either the [Issues](https://github.com/Gi-z/CSIKit/issues)
or email me: g.r.forbes@rgu.ac.uk.

### Contents:

* [Overview](#overview)
* [Imports](#imports)
* [Using Scenario Plotter](#using-scenarioplotter)
* [Save to PDF](#save-to-pdf)
* [Custom Metrics](#custom-metrics)
* [Custom Graphs](#custom-graphs)

## Overview <a name="overview"></a>

The `ScenarioPlotter` class can be used to generate many plots for a given CSI file,
by providing a list of metrics and desired plot types. These plots can then be rendered onscreen or to PDF.

## Imports <a name="imports"></a>

In [29]:
from CSIKit.visualization.graph import *
from CSIKit.visualization.metric import *
from CSIKit.visualization.plot_scenario import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Using ScenarioPlotter <a name="using-scenarioplotter"></a>

`ScenarioPlotter`'s constructor takes two parameters:

1. `scenario_name` (str): This name is used in generating the filename when saving plots to PDF.
2. `plot_impls` (List): A list of tuples containing a Metric and Graph.

Once a `ScenarioPlotter` object has been instantiated, CSI measurement files can be added with the
`add_measurements_files(files)` function.
This function takes `files`: a dict of names (keys) and paths to CSI files (values).
The selected plots can then be rendered onscreen using `show()`.

In this example, we'll generate plots for all available metrics to demonstrate them.

Metrics:

 - RSS: Received Signal Strength.
 - RSS_per_Antenna: RSS for each antenna stream.
 - RSSI: Received Signal Strength Indicator.
 - RSSI_per_Antenna: RSSI for each antenna stream.
 - Noise: Noise floor in dB.
 - SNR: Signal-to-noise ratio.
 - Amplitude_Sum: CSI Amplitude summed across each subcarrier and antenna stream.
 - Phase_Diff_Std_err: Standard Deviation of Phase Difference across each antenna stream.
 - Datarate: 802.11 datarate for each CSI frame.
 - AGC: Automatic Gain Control setting.
 - CSI_Matrix_Phase_Diff_1_2: Phase difference across two antenna streams.

Graphs:

 - PlotCandle: Candle plot.
 - PlotCandleTuple: Multiple associated candle plots.
 - PlotCandleTuple_Phase: As above, tuned for rendering phase rather than amplitude.
 - PlotBox: Box plot.
 - PlotColorMap: 2d image plot.
 - PlotColorMap_Phase: As above, tuned for rendering phase rather than amplitude.

In [ ]:
PLOT_CLASSES = [
    (RSS,                   PlotCandle),
    (RSS_per_Antenna,       PlotCandleTuple),
    (RSSI,                  PlotCandle),
    (RSSI,                  PlotBox),
    (RSSI_per_Antenna,      PlotCandleTuple),
    (Noise,                 PlotCandle),
    (SNR,                   PlotCandle),
    (Amplitude_Sum,         PlotCandle),
    (Phase_Diff_Std_err,    PlotCandleTuple_Phase),
    (Datarate,              PlotCandle),
    (AGC,                   PlotCandle),
    (CSI_Matrix_Phase_Diff_1_2,                   PlotColorMap_Phase),
]

FILES = {
    "5 Meters"      :"../CSIKit/visualization/example_dat/los_5.dat",
    "40 Meters"     :"../CSIKit/visualization/example_dat/los_40.dat",
    "100 Meters"    :"../CSIKit/visualization/example_dat/los_100.dat",
}

scenario = ScenarioPlotter("name_to_save_pdf", PLOT_CLASSES)
scenario.add_measurements_files(FILES)
scenario.show()

## Save to PDF <a name="save-to-pdf"></a>

Alternatively, you can save plots to PDF using `save(folder="OUTPUT_FOLDER_LOCATION")`.

In [ ]:
scenario.save(folder="output")

## Custom Metrics <a name="custom-metrics"></a>

If the metrics included do not fit your specifications, custom metrics can be written with relative ease.
Inheriting the `Metric` class, simply define the following functions:

 - `notice`: Generating and returning the value represented by your metric, for a given CSI frame.
 - `get_name`: Returning the metric name.
 - `get_unit`: Returning the units for your metric.

In [ ]:
class Noise(Metric):
    def notice(self, entry:CsiEntry):
        return entry.noise
    def get_name(self):
        return "Noise"
    def get_unit(self):
        return "dBm"

<a name="custom-graphs"></a>
## Custom Graphs

To implement your own graph, implement the show method at Graph and use the given matplotlib **axes**
and values with the following shape:
``` python
{
    "<measurement_name1>" : entires:list[Frames],
    "<measurement_name2>" : entires:list[Frames],
    (...)
}
```

Shown below is an example of a Graph implementation for rendering 2d colormaps.

In [ ]:
class PlotColorMap(Graph):

    def __init__(self, metric):
        super().__init__(metric)
        self.vmin = None
        self.vmax = None
        self.cmap = plt.cm.plasma
        self.color_legend = True

    def _plot_axes(self,  values_per_measurement):

        for measurement_name in values_per_measurement:
            axes = self._create_new_ax()
            amplitude_per_sub = values_per_measurement[measurement_name]
            amplitude_per_sub = np.matrix(np.array(amplitude_per_sub))

            cmap = self.cmap
            if not self.vmin is None and not self.vmax is None:
                pcmap = axes.pcolormesh(amplitude_per_sub, cmap=cmap,  vmin=self.vmin, vmax=self.vmax,rasterized=True)
            else:
                pcmap = axes.pcolormesh(amplitude_per_sub, cmap=cmap, rasterized=True)

            if self.color_legend:
                plt.colorbar(pcmap, ax=axes)

            axes.set_xlabel("Subcarrier")
            axes.set_ylabel('Measurement')
            plt.show()